In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Psoriasis/GSE175438'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"MicroRNA expression data from skin samples of adult atopic dermatitis and psoriasis patients and healthy controls"
!Series_summary	"Atopic dermatitis (AD) and psoriasis vulgaris (PV) affect up to 3-5% of adults in developed countries and severely impair their quality of life thorough adulthood. MicroRNAs contribute to either the development or regulation of several diseases and are present in tissue."
!Series_summary	"Although several miRNAs have been associated with AD or PV, to our knowledge, scarce studies have been published to date of miRNA profiling comparing AD and PV patients."
!Series_overall_design	"We conducted a miRNA profiling analysis of skin samples from adult AD and PV patients and control individuals."
Sample Characteristics Dictionary:
{0: ['subject status: atopic dermatitis adult patient', 'subject status: healthy control adult', 'subject status: psoriasis adult patient'], 1: ['tissue: skin']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Determine if the dataset contains gene expression data
if 'subject status: psoriasis adult patient' in sample_characteristics_dict[0]:
    is_gene_available = True
else:
    is_gene_available = False

# Check availability of 'Psoriasis' trait
if 'subject status: psoriasis adult patient' in sample_characteristics_dict[0]:
    trait_row = 0
else:
    trait_row = None
    
# Age information not available in the sample characteristics
age_row = None

# Gender information not available in the sample characteristics
gender_row = None

# Define the conversion function for 'Psoriasis' trait
def convert_trait(value):
    if 'psoriasis' in value.lower():
        return 1
    elif 'healthy' in value.lower() or 'atopic dermatitis' in value.lower():
        return 0
    else:
        return None

convert_age = None  # Age conversion function not needed
convert_gender = None  # Gender conversion function not needed

# Save cohort information
save_cohort_info('GSE175438', './preprocessed/Psoriasis/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save clinical feature data if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Psoriasis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Psoriasis/trait_data/GSE175438.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5333593': [0], 'GSM5333594': [0], 'GSM5333595': [0], 'GSM5333596': [0], 'GSM5333597': [0], 'GSM5333598': [0], 'GSM5333599': [0], 'GSM5333600': [0], 'GSM5333601': [0], 'GSM5333602': [0], 'GSM5333603': [0], 'GSM5333604': [0], 'GSM5333605': [0], 'GSM5333606': [0], 'GSM5333607': [0], 'GSM5333608': [0], 'GSM5333609': [0], 'GSM5333610': [0], 'GSM5333611': [0], 'GSM5333612': [0], 'GSM5333613': [1], 'GSM5333614': [1], 'GSM5333615': [1], 'GSM5333616': [1], 'GSM5333617': [1], 'GSM5333618': [1], 'GSM5333619': [1], 'GSM5333620': [1], 'GSM5333621': [1], 'GSM5333622': [1], 'GSM5333623': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['20500112', '20500113', '20500114', '20500115', '20500116', '20500117',
       '20500118', '20500119', '20500120', '20500121', '20500122', '20500123',
       '20500124', '20500125', '20500126', '20500127', '20500128', '20500129',
       '20500130', '20500131'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['20500000', '20500001', '20500002', '20500003', '20500004'], 'Accession_ID': ['MIMAT0000001_st', 'MIMAT0015091_st', 'MIMAT0000002_st', 'MIMAT0015092_st', 'MIMAT0020301_st'], 'Accession': ['MIMAT0000001', 'MIMAT0015091', 'MIMAT0000002', 'MIMAT0015092', 'MIMAT0020301'], 'Transcript ID(Array Design)': ['cel-let-7-5p', 'cel-let-7-3p', 'cel-lin-4-5p', 'cel-lin-4-3p', 'cel-miR-1-5p'], 'Sequence Type': ['miRNA', 'miRNA', 'miRNA', 'miRNA', 'miRNA'], 'Species Scientific Name': ['Caenorhabditis elegans', 'Caenorhabditis elegans', 'Caenorhabditis elegans', 'Caenorhabditis elegans', 'Caenorhabditis elegans'], 'Alignments': ['X:14744165-14744186 (-)', 'X:14744123-14744147 (-)', 'II:5902246-5902266 (+)', 'II:5902285-5902305 (+)', 'I:6172718-6172739 (-)'], 'Sequence Length': ['22', '25', '21', '21', '22'], 'Sequence': ['UGAGGUAGUAGGUUGUAUAGUU', 'UGAACUAUGCAAUUUUCUACCUUAC', 'UCCCUGAGACCUCAAGUGUGA', 'ACACCUGGGCUCUCCGGGUAC', 'CAUACUUCCUUACAUGCCCAUA'], 'Genome Context': [

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Key corresponding to probe IDs (identifiers) and gene symbols in the gene annotation dictionary
identifier_key = 'ID'
gene_symbol_key = 'Transcript ID(Array Design)'

# 2. Get the dataframe storing the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to obtain the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print a preview of the resulting gene expression dataframe to verify the mapping
print("Mapped gene expression data preview:")
print(preview_df(gene_data))


Mapped gene expression data preview:
{'GSM5333593': [0.14, 0.21000000000000002, 0.06, 0.1, 0.030000000000000002], 'GSM5333594': [0.24, 0.16, 0.14, 0.11, 0.035], 'GSM5333595': [0.3, 0.045, 0.05, 0.12, 0.075], 'GSM5333596': [0.15, 0.16, 0.02, 0.16, 0.125], 'GSM5333597': [0.37, 0.125, 0.01, 0.19, 0.125], 'GSM5333598': [0.16, 0.08499999999999999, 0.03, 0.04, 0.155], 'GSM5333599': [0.15, 0.245, 0.16, 0.1, 0.055], 'GSM5333600': [0.17, 0.02, 0.15, 0.07, 0.035], 'GSM5333601': [0.23, 0.0, 0.12, 0.1, 0.16999999999999998], 'GSM5333602': [0.16, 0.125, 0.16, 0.02, 0.11], 'GSM5333603': [0.07, 0.16, 0.06, 0.05, 0.065], 'GSM5333604': [0.16, 0.135, 0.11, 0.2, 0.05], 'GSM5333605': [0.03, 0.265, 0.16, 0.07, 0.205], 'GSM5333606': [0.12, -0.02, 0.16, 0.0, 0.16], 'GSM5333607': [0.04, 0.125, 0.01, 0.1, 0.135], 'GSM5333608': [0.27, 0.18, 0.14, 0.04, 0.03], 'GSM5333609': [-0.07, 0.15, 0.0, 0.1, -0.075], 'GSM5333610': [0.06, 0.21000000000000002, 0.03, 0.09, 0.1], 'GSM5333611': [0.06, 0.15000000000000002, 0.1, 0

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Psoriasis/gene_data/GSE175438.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Psoriasis')

# Assign trait_biased to trait_biased to avoid undefined variable error
trait_biased = trait_biased

# 4. Save the cohort information.
save_cohort_info('GSE175438', './preprocessed/Psoriasis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Psoriasis/GSE175438.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Psoriasis', the least common label is '1.0' with 11 occurrences. This represents 35.48% of the dataset.
The distribution of the feature 'Psoriasis' in this dataset is fine.

